# Individual differences (part 3)

In [ ]:
%load_ext lab_black
import pandas as pd
import numpy as np
import altair as alt
import helper
from altair.expr import datum
from IPython.display import clear_output
from tqdm import tqdm

alt.data_transformers.disable_max_rows()

### Import parsed datafile

In [ ]:
sim2 = helper.parse_from_file("../sims/part3_1750.csv")
sim2 = helper.SimResults(sim2)
sim2.select_by_cond(["HF_INC", "NW_UN"])
sim2.df.columns

### Heatmap for overview

In [ ]:
sim2.plot_heatmap_wadv(mode="dev").save("wadv_development.html")
sim2.plot_heatmap_wadv(mode="per").save("wadv_performance.html")

### Plot main effect in  performance space

In [ ]:
sim2.plot_performance_multiline(var="p_noise", color="reds").save(
    "performance_pnoise_main.html"
)
sim2.plot_performance_multiline(var="hidden_units", color="blues").save(
    "performance_hidden_main.html"
)
sim2.plot_performance_multiline(var="learning_rate", color="greens").save(
    "performance_epsilon_main.html"
)

### Plot interactions in performance space (P-noise as focus)

In [ ]:
plot = alt.hconcat()
for h in tqdm(np.sort(sim2.df.hidden_units.unique())[::-1]):
    plot_row = alt.vconcat()
    for l in np.sort(sim2.df.learning_rate.unique())[::-1]:
        tmp = helper.parse_from_file("../sims/part3_1750.csv")
        tmp = helper.SimResults(tmp)
        tmp.select_by_cond(["HF_INC", "NW_UN"], verbose=False)
        tmp.select_by_control(hidden_units=[h], learning_rate=[l], verbose=False)
        plot_row |= tmp.plot_performance_multiline(
            var="p_noise", color="reds"
        ).properties(title=f"hidden = {h}, epsilon = {l}")

    plot &= plot_row

plot.save("performance_pnoise_interaction.html")

### Plot interactions in performance space (Hidden as focus)

In [ ]:
plot = alt.hconcat()
for p in tqdm(np.sort(sim2.df.p_noise.unique())):
    plot_row = alt.vconcat()
    for l in np.sort(sim2.df.learning_rate.unique())[::-1]:
        tmp = helper.parse_from_file("../sims/part3_1750.csv")
        tmp = helper.SimResults(tmp)
        tmp.select_by_cond(["HF_INC", "NW_UN"], verbose=False)
        tmp.select_by_control(p_noise=[p], learning_rate=[l], verbose=False)
        plot_row |= tmp.plot_performance_multiline(
            var="hidden_units", color="blues"
        ).properties(title=f"p-noise = {p}, epsilon = {l}")

    plot &= plot_row

plot.save("performance_hidden_interaction.html")

### Plot interactions in performance space (Epsilon as focus)

In [ ]:
plot = alt.hconcat()
for p in tqdm(np.sort(sim2.df.p_noise.unique())):
    plot_row = alt.vconcat()
    for h in np.sort(sim2.df.hidden_units.unique())[::-1]:
        tmp = helper.parse_from_file("../sims/part3_1750.csv")
        tmp = helper.SimResults(tmp)
        tmp.select_by_cond(["HF_INC", "NW_UN"], verbose=False)
        tmp.select_by_control(p_noise=[p], hidden_units=[h], verbose=False)
        plot_row |= tmp.plot_performance_multiline(
            var="learning_rate", color="greens"
        ).properties(title=f"p-noise = {p}, hidden = {h}")
        clear_output(wait=True)

    plot &= plot_row

plot.save("performance_epsilon_interaction.html")